In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import is_bfloat16_supported
from datasets import load_dataset
from unsloth.chat_templates import train_on_responses_only
from PIL import Image
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp2/howard/venv_manager/vl-sft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 8192
dtype = None
load_in_4bit = True

model_name = "/tmp2/share_data/google--gemma-3-12b-it"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    attn_implementation="eager",
)


==((====))==  Unsloth 2025.12.5: Fast Gemma3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.91s/it]


In [ ]:
# Enable vision layer fine-tuning for image input
model = FastLanguageModel.get_peft_model(
    model,
    finetune_vision_layers=False,   # Turn ON for vision input!
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    random_state=3407,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [5]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3",
)

### Dataset

In [11]:

# Load dataset - expected format with images
raw_train_dataset, raw_val_dataset = load_dataset(
    "json",
    data_files="data/input/example.json",
    split=["train[:90%]", "train[90%:]"],
)


Generating train split: 7 examples [00:00, 202.55 examples/s]


In [12]:

instruction = "請給我 OCR 結果"

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image_path"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["ocr_text"]} ]
        },
    ]
    return { "messages" : conversation }

train_dataset = [convert_to_conversation(sample) for sample in raw_train_dataset]
val_dataset = [convert_to_conversation(sample) for sample in raw_val_dataset]

In [13]:
train_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '請給我 OCR 結果'},
    {'type': 'image',
     'image': 'data/pdf2png/mohw/(檔案下載)衛部心字第1131763377號發布令/(檔案下載)衛部心字第1131763377號發布令_page_0001.png'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '檔號：保存年限：\n衛生福利部 令\n發文日期：中華民國114年1月17日\n發文字號：衛部心字第1131763377號\n附件：「精神疾病嚴重病人強制處置費用標準」條文1份\n\n訂定「精神疾病嚴重病人強制處置費用標準」。\n附「精神疾病嚴重病人強制處置費用標準」部長 邱泰源\n\n第1页 共1页'}]}]}

In [14]:
from unsloth import FastVisionModel

FastVisionModel.for_inference(model) # Enable for inference!

image = raw_train_dataset[0]["image_path"]
instruction = "請給我 OCR 結果"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
# 加入一些 tag
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Here's the OCR result from the image you sent:

**楷 號:**
保存年限:

**衛生福利部 令**

發文日期：中華民國114年1月17日
發文字號：衛部心字第1131763377號
附件： 「精神疾病嚴重病人強制處置費用標準」 條文1份

訂定「精神疾病嚴重病人強制處置費用標準」。

附「精神疾病嚴重病人強制處置費用標準」

**部長 邱 泰 源**

第1頁 共1


### Training

In [15]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = train_dataset,
    eval_dataset= val_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        num_train_epochs=5,
        eval_strategy="steps",
        save_strategy="steps",
        save_steps=1000,
        metric_for_best_model="eval_loss",
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "models/vision_1215",
        report_to = "none",

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = max_seq_length,
    ),
)

In [16]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.542 GB.
11.254 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6 | Num Epochs = 5 | Total steps = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 68,456,448 of 12,255,781,488 (0.56% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
1,2.098100,2.065999
2,2.098100,1.423766
3,1.430300,1.058661
4,1.058100,0.875768
5,0.867000,0.705856
6,0.703300,0.588672
7,0.593200,0.499540
8,0.497100,0.442612
9,0.443600,0.419970
10,0.419000,0.412737


Unsloth: Not an error, but Gemma3ForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [18]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

27.1876 seconds used for training.
0.45 minutes used for training.
Peak reserved memory = 11.254 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 47.804 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Inference

In [19]:
FastVisionModel.for_inference(model) # Enable for inference!

image = raw_train_dataset[0]["image_path"]
instruction = "請給我 OCR 結果"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

檔號：保存年限：
衛生福利部 令
發文日期：中華民國114年1月17日
發文字號：衛部心字第1131763377號
附件：「精神疾病嚴重病人強制處置費用標準」條文1份
訂定「精神疾病嚴重病人強制處置費用標準」。
附「精神疾病嚴重病人強制處置費用標準」部長 邱泰源
第1页 共1页<end_of_turn>


### Saving for VLLM

In [20]:
output_dir = "/tmp2/howard/vl-sft-ocr/models/merged--gemma-3-12b-sft_input_size=500"

model.save_pretrained_merged(output_dir, tokenizer, save_method = "merged_16bit",)

Detected local model directory: /tmp2/share_data/google--gemma-3-12b-it
Copied tokenizer.model from local model directory
Found HuggingFace hub cache directory: /home/howard/.cache/huggingface/hub


Unsloth: Preparing safetensor model files:  20%|██████████████████████████▏                                                                                                        | 1/5 [00:01<00:05,  1.37s/it]

Copied model-00003-of-00005.safetensors from local model directory


Unsloth: Preparing safetensor model files:  40%|████████████████████████████████████████████████████▍                                                                              | 2/5 [00:11<00:19,  6.49s/it]

Copied model-00002-of-00005.safetensors from local model directory


...

Copied model-00001-of-00005.safetensors from local model directory


Unsloth: Preparing safetensor model files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:08<00:00, 37.69s/it]


Copied model-00005-of-00005.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:06<00:00, 37.22s/it]


Unsloth: Merge process complete. Saved to `/tmp2/howard/vl-sft-ocr/models/merged--gemma-3-12b-sft_input_size=500`
